# Module 2: Multi-Agent - Agent as Tool Pattern

## Creating an Evaluator Agent for UniTok Posts

In this notebook, we'll build on Module 1 by implementing an evaluator agent that ensures posts adhere to brand guidelines before publishing. We'll use the "Agent as Tool" pattern, where our post generator agent will use the evaluator agent as a specialized tool.

## The "Agent as Tool" Pattern

The "Agent as Tool" pattern is an architectural approach where specialized AI agents are wrapped as callable functions (tools) that can be used by other agents. This creates a hierarchical structure where:

1. A primary agent handles the main task and determines when to call specialized agents
2. Specialized "tool agents" perform domain-specific tasks when called by the primary agent

### Key Benefits

- **Separation of concerns**: Each agent has a focused area of responsibility
- **Specialized expertise**: Agents can be optimized for specific tasks with tailored system prompts
- **Hierarchical delegation**: Clear chain of command for complex tasks
- **Modular architecture**: Specialists can be added, removed, or modified independently
- **Improved quality**: Multiple agents can provide checks and balances
- **Feedback loops**: Agents can learn from each other's outputs

In our case, we'll create:
- A content evaluator agent specialized in ensuring posts adhere to brand guidelines
- A feedback loop where the post generator can revise content based on evaluator feedback

## Setup and Prerequisites

First, let's install the required packages:

In [ ]:
!pip install --upgrade strands-agents requests boto3

Now let's import the necessary libraries:

In [ ]:
import json
import requests
from strands import Agent, tool
from strands.models import BedrockModel

## Creating the Evaluator Agent

First, let's create an evaluator agent that specializes in ensuring posts adhere to Unicorn Rentals' brand guidelines.

In [ ]:
# Define the system prompt for our evaluator agent
EVALUATOR_SYSTEM_PROMPT = """
You are a specialized content evaluator for Unicorn Rentals, a company that offers unicorns for rent that kids and grown-ups can play with.

Your task is to evaluate social media posts for UniTok, our unicorn-themed social media platform, and ensure they adhere to our brand guidelines.

Brand Guidelines for Unicorn Rentals:
1. Content must be family-friendly and positive
2. Posts should highlight the magical experience of spending time with unicorns
3. When mentioning our new color selection feature, it should be accurate (colors: pink, blue, purple, green, yellow, and rainbow)
4. Our brand voice is magical, playful, and family-friendly
5. Posts should be between 50-200 characters for optimal engagement
6. Emojis should be used sparingly but effectively
7. Content should appeal to our target audience: families with children, fantasy enthusiasts, and event planners

For each post you evaluate:
1. Check if it adheres to ALL brand guidelines
2. Provide a clear APPROVED or REJECTED decision
3. If rejected, provide specific feedback on what needs to be improved
4. If approved, provide a brief explanation of why it meets our guidelines

Your evaluation should be thorough but concise.
"""

# Initialize the model
model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
)

# Create the evaluator agent
evaluator_agent = Agent(
    model=model,
    system_prompt=EVALUATOR_SYSTEM_PROMPT,
)

Let's test our evaluator agent with a few sample posts to see how it evaluates them:

In [ ]:
# Test with a good post
good_post = "✨ Rainbow or Pink? Now YOU choose your unicorn's color! Our new feature lets kids pick their magical companion in their favorite shade. Book your enchanted playdate today! 🦄"

evaluation = evaluator_agent(f"Please evaluate this social media post: '{good_post}'")
print(evaluation)

In [ ]:
# Test with a post that violates guidelines
bad_post = "Our unicorns are now available in red, orange and black colors! Adults will love the scary black unicorn for Halloween parties! Book now for a wild night of unicorn fun!"

evaluation = evaluator_agent(f"Please evaluate this social media post: '{bad_post}'")
print(evaluation)

## Creating the Evaluator Tool

Now, let's wrap our evaluator agent as a tool that can be used by other agents. This is the key step in implementing the "Agent as Tool" pattern.

In [ ]:
@tool
def evaluate_post(content: str) -> str:
    """
    Evaluate a social media post for adherence to Unicorn Rentals brand guidelines.
    
    Args:
        content (str): The text content of the post to evaluate.
        
    Returns:
        str: Evaluation result with APPROVED or REJECTED decision and specific feedback.
    """
    try:
        # Create a new evaluator agent instance
        evaluator = Agent(
            model=model,
            system_prompt=EVALUATOR_SYSTEM_PROMPT,
        )
        
        # Call the evaluator agent to evaluate the post
        response = evaluator(f"Please evaluate this social media post: '{content}'")
        
        return str(response)
    except Exception as e:
        return f"Error in post evaluation: {str(e)}"

Let's test our evaluator tool directly:

In [ ]:
test_post = "Introducing our new unicorn colors! Now available in pink, blue, purple, green, yellow, and rainbow! 🦄✨"

evaluation_result = evaluate_post(test_post)

## Creating the Publishing Tool

Let's recreate the publishing tool from Module 1:

In [ ]:
# Define the API endpoint
# In a real deployment, this would be the actual API Gateway URL from CDK output
API_ENDPOINT = "https://08pzccde2k.execute-api.us-east-1.amazonaws.com/prod/posts"  # Replace with your actual API endpoint

@tool
def publish_post(content: str, author: str = "Unicorn Rentals", unicorn_color: str = "rainbow", image_url: str = None) -> str:
    """
    Publish a post to the UniTok social media platform.
    
    Args:
        content (str): The text content of the post.
        author (str, optional): The author of the post. Defaults to "Unicorn Rentals".
        unicorn_color (str, optional): The color of the unicorn. Choose from: pink, blue, purple, green, yellow, or rainbow. Defaults to "rainbow".
        image_url (str, optional): URL to an image to include with the post. Defaults to None.
        
    Returns:
        str: A message indicating the post was published successfully, or an error message.
    """
    try:
        # Prepare the post data
        post_data = {
            "content": content,
            "author": author,
            "unicornColor": unicorn_color,
        }
        
        if image_url:
            post_data["imageUrl"] = image_url
            
        # Send the post to the API
        response = requests.post(API_ENDPOINT, json=post_data)
        
        # Check if the request was successful
        if response.status_code == 201:
            post_id = response.json().get("postId")
            return f"Post published successfully! Post ID: {post_id}"
        else:
            return f"Failed to publish post. Status code: {response.status_code}, Response: {response.text}"
            
    except Exception as e:
        return f"Error publishing post: {str(e)}"

## Creating the Enhanced Post Generator Agent

Now, let's create an enhanced post generator agent that uses the evaluator tool to ensure posts adhere to brand guidelines before publishing.

In [ ]:
# Define the system prompt for our enhanced post generator agent
ENHANCED_GENERATOR_PROMPT = """
You are a creative social media manager for Unicorn Rentals, a company that offers unicorns for rent that kids and grown-ups can play with.

Your task is to create engaging social media posts for UniTok, our unicorn-themed social media platform.
Before publishing, you must use the evaluate_post tool to ensure your content adheres to our brand guidelines.

Process for creating and publishing posts:
1. Generate a creative post based on the user's request
2. Use the evaluate_post tool to check if it meets our brand guidelines
3. If the post is REJECTED, revise it based on the feedback and evaluate again
4. Once the post is APPROVED, publish it using the publish_post tool

Important information about Unicorn Rentals:
- We offer unicorns in various colors: pink, blue, purple, green, yellow, and rainbow (our most popular)
- Our new product feature allows customers to pick their favorite color unicorn to rent
- Our target audience includes families with children, fantasy enthusiasts, and event planners
- Our brand voice is magical, playful, and family-friendly

When creating posts:
- Keep content family-friendly and positive
- Highlight the magical experience of spending time with unicorns
- Mention the new color selection feature when appropriate
- Use emojis sparingly but effectively
- Keep posts between 50-200 characters for optimal engagement

Always show your thought process when creating posts, evaluating them, and making revisions.
"""

# Create the enhanced post generator agent
enhanced_post_generator = Agent(
    model=model,
    system_prompt=ENHANCED_GENERATOR_PROMPT,
    tools=[evaluate_post, publish_post],
)



## Testing the Multi-Agent System

Let's test our enhanced post generator agent to see how it creates posts, evaluates them, and revises them if necessary before publishing.

In [ ]:
response = enhanced_post_generator("Create a post announcing our new feature that lets customers pick their favorite unicorn color.")
print(response)

Let's try another example that might require revision:

In [ ]:
enhanced_post_generator = Agent(
    model=model,
    system_prompt=ENHANCED_GENERATOR_PROMPT,
    tools=[evaluate_post, publish_post],
)
response = enhanced_post_generator("Create a post about our unicorns for an adult Halloween party. Make it really scary!")

## Viewing Posts on the UniTok Website

To view the posts that our enhanced agent has created and published:

1. Find the **UniTokUrl** from your CDK deployment output. It should look something like: `https://d123abc456def.cloudfront.net`
2. Open this URL in your web browser
3. You should see the posts that our agent has created and published, displayed in reverse chronological order

Notice how the quality of posts has improved with the evaluator agent ensuring adherence to brand guidelines.

## Analyzing Tool Usage

Let's examine how the agent used our custom tools:

In [ ]:
for message in enhanced_post_generator.messages:
    for content_item in message.get('content', []):
        if isinstance(content_item, dict) and 'toolUse' in content_item:
            tool_use = content_item['toolUse']
            print(f"Tool: {tool_use['name']}")
            print(f"Input: {tool_use['input']}")
            print("---")

## Advantages of the Agent as Tool Pattern

Through this implementation, we've demonstrated several key advantages of the "Agent as Tool" pattern:

1. **Separation of concerns**:
   - The post generator focuses on creativity and content creation
   - The evaluator focuses on ensuring brand guideline adherence

2. **Specialized expertise**:
   - Each agent has a tailored system prompt optimized for its specific task
   - The evaluator has detailed knowledge of brand guidelines
   - The generator has creative freedom within defined constraints

3. **Feedback loops**:
   - The generator can learn from the evaluator's feedback
   - Content quality improves through iterative refinement

4. **Modular architecture**:
   - We can modify the evaluator's guidelines without changing the generator
   - We could add additional specialized agents (e.g., a hashtag generator, image suggester)

5. **Quality control**:
   - Posts are vetted before publishing
   - Brand consistency is maintained

This pattern mimics human team dynamics, where a creative writer might submit content to an editor for review before publication. The multi-agent approach allows for checks and balances that improve the overall quality of the output.

## Conclusion

In this notebook, we've successfully implemented a multi-agent system using the "Agent as Tool" pattern. We've created:

1. An evaluator agent specialized in ensuring posts adhere to brand guidelines
2. An enhanced post generator that uses the evaluator as a tool
3. A feedback loop where posts are revised based on evaluation results

This approach demonstrates how multiple specialized agents can work together to produce higher quality results than a single agent working alone. The modular architecture also makes the system more maintainable and extensible.

In the next module, we'll build on this foundation by adding human-in-the-loop approval workflows.